In [1]:
from fastdtw import fastdtw
from sklearn.preprocessing import normalize
import numpy as np
import core.yahoo_core as yh
import cufflinks as cf
import yfinance as yf
import utils.proxy_utils as pu
from utils.cf_util import xaxes
import datetime
from core.number_regular import find_similar_sequences
import pandas as pd
pu.proxy()
cf.set_config_file(offline=True, world_readable=True)

In [121]:
begin = datetime.date(2000, 1, 1)
end = datetime.datetime.now().date()
data = yf.download('AMZN', begin.strftime("%Y-%m-%d"), datetime.date(2023, 6, 1).strftime("%Y-%m-%d"))
s_data = yf.download('QQQ', datetime.date(2023, 6, 1).strftime("%Y-%m-%d"), end)
qf = cf.QuantFig(data, title='QQQ', legend='top', name='QQQ')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [122]:
fig = qf.figure(up_color='green', down_color='red')
fig.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(begin, end, data))  # 剔除所有非交易日
    ]
)
fig.show()

In [123]:
source = data[['Open','Close','High','Low']].values.tolist()
regular = s_data[['Open','Close','High','Low']].values.tolist()

In [124]:
all_re = find_similar_sequences(source, regular, 0.2)
len(all_re)

35

In [120]:
df = pd.DataFrame(all_re[7]['k'], columns=['Open','Close','High','Low'])
r_qf = cf.QuantFig(df, title='QQQ', legend='top', name='QQQ')
r_qf.figure(up_color='green', down_color='red')

In [102]:
# df_spy = pd.DataFrame(regular, columns=['Open','Close','High','Low'])
r_qf = cf.QuantFig(s_data, title='SPY', legend='top', name='SPY')
r_qf.figure(up_color='green', down_color='red')
fig = r_qf.figure(up_color='green', down_color='red')
fig.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(begin, end, s_data))  # 剔除所有非交易日
    ]
)
fig.show()

In [104]:
df = data.iloc[all_re[0]['index'] : all_re[0]['index'] + len(s_data) + 20]
r_qf = cf.QuantFig(df, title='QQQ', legend='top', name='QQQ')
r_qf.figure(up_color='green', down_color='red')
fig = r_qf.figure(up_color='green', down_color='red')
fig.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(begin, end, df))  # 剔除所有非交易日
    ]
)
fig.show()

In [105]:
p = data.iloc[all_re[0]['index'] + len(s_data): all_re[0]['index'] + len(s_data) + 20]
open = p.iloc[0]['Open']
close = s_data.iloc[-1]['Close']
xi = close / open
p = p * xi
p = p.reset_index(drop=False)
today = pd.to_datetime('today').normalize()
p['Date'] = today + (p['Date'] - p['Date'].min())
end_date = p.iloc[-1]['Date'].to_pydatetime().date()
p = p.set_index('Date', drop=False)

pdf = pd.concat([s_data, p], axis=0)
r_qf = cf.QuantFig(pdf, title='SPY_Predict', legend='top', name='SPY_Predict')
r_qf.figure(up_color='green', down_color='red')
fig = r_qf.figure(up_color='green', down_color='red')
fig.update_xaxes(
    rangebreaks=[
        dict(values=xaxes(begin, end_date, pdf))  # 剔除所有非交易日
    ]
)
fig.show()

In [35]:
p = data.iloc[all_re[0]['index'] + len(s_data): all_re[0]['index'] + len(s_data) + 20]

In [39]:
p.iloc[0]['Open']

68.25

In [65]:
p = data.iloc[all_re[0]['index'] + len(s_data): all_re[0]['index'] + len(s_data) + 20]
open = p.iloc[0]['Open']
close = s_data.iloc[-1]['Close']
xi = close / open
p = p * xi
p = p.reset_index(drop=False)
today = pd.to_datetime('today').normalize()
p['Date'] = today + (p['Date'] - p['Date'].min())
end_date = p.iloc[-1]['Date']

In [68]:
end_date.to_pydatetime()

datetime.datetime(2023, 8, 19, 0, 0)

KeyError: 'Date'